In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

# Naive Baye's Classifier

Naive Baye's classifier is based on Baye's theorem and is used for classification problems. For instance in NLP text classification: topic modeling, sentiment analysis, spam detection etc

## Bayes' theorem:

Partition of set $X$ on subsets $\{A_i \subset X|I \in I\}$ is $X = \bigcup_{i \in I}A_i$ and $A_i \cap A_j = \emptyset$ for every pair $i,j \in I$

For each subset $A \subset X$ we have partition $X = A \cup A^c$

#### Total probability theorem:
For every partition $A_1, A_2 \dots, A_k$ of $\Omega$ and event $B \subset \Omega$:
$$P(B) = \sum_{i=1}^{k}P(B|A_i)P(A_i)$$

#### Theorem (Bayes' theorem):
Let $A_1, A_2 \dots, A_k$ be a partition of $\Omega$ such that $P(A_i) > 0$ for each $i \in \{1, 2, \dots, k\}$. Then for $B \subset \Omega$ event, such that $P(B) > 0$, for each $i \in \{1, 2, \dots, k\}$:
$$
P(A_i|B) = \frac{P(B|A_i)P(A_i)}{\sum_{j=1}^{k}P(B|A_j)P(A_j)}
$$

#### Note:
We call $P(A_i)$ the prior probability and $P(A_i|B)$ the posterior probability

For the events $A$ and $B$ such that $P(B) \gt 0$ we have:
$$
P(A|B) = \frac{P(B|A)P(A)}{P(B)}
$$
<br>
We can consider the partition of $\Omega$ on $A$ and $A^c$, the from Byes' theorem we have:
$$
P(A|B) = \frac{P(B|A)P(A)}{P(B|A)P(A) + P(B|A^c)P(A^c)} = \text{(by the total probability) }\frac{P(B|A)P(A)}{P(B)}
$$

#### Example:
Divide emails $A_1 = \text{"spam"}$, $A_2 = \text{"low priority"}$ and $A_3 = \text{"high priority"}$ and let: $P(A_1) = 0.7$, $P(A_2) = 0.2$ and $P(A_3) = 0.1$. ($P(A_1) + P(A_2) + P(A_3) = 0.7 + 0.2 + 0.1 = 1$)
<br>
Let $B$ be the event that email contains the word "free" and we know from previous experience that: $P(B|A_1) = 0.9$, $P(B|A_2) = 0.01$ and $P(B|A_3) = 0.01$.
<br>
If we receive the email with word "free" in it, what is the probability, that this email is spam?
From Bayes' theorem:
$$
P(A_1|B) = \frac{P(B|A_1)P(A_1)}{P(B|A_1)P(A_1) + P(B|A_2)P(A_2) + P(B|A_3)P(A_3)} = \frac{0.9 \cdot 0.7}{0.9 \cdot 0.7 + 0.01 \cdot 0.2 + 0.01 \cdot .01} = 0.995
$$

## Multi-dimensional case

#### Example Golf and Weather

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

In [ ]:
path = Path('data')
nb = path / 'naive_bayes'
golf_csv = nb / 'golf.csv'

In [ ]:
def strip_txt(txt:str) -> str:
    return txt.replace("'", '').strip() if txt else txt

In [ ]:
df = pd.read_csv(golf_csv, converters={'outlook':strip_txt,
                                       'temp': strip_txt,
                                       'humidity': strip_txt,
                                       'wind': strip_txt,
                                       'label': strip_txt})
df

Let's calculate each Label probabilities
<br>
$P(\text{"Yes"}) = 9/14$ and $P(\text{"No"}) = 5/14$

Let't conditional probability of outlook 'Sunny' feature with respect of labels
<br>
$P(\text{"Yes"}|\text{"Sunny"}) = 2/9$ and $P(\text{"No"}|\text{"Sunny"}) = 3/5$


Let't conditional probability of outlook 'Overcast' feature with respect of labels
<br>
$𝑃("Yes"|"Overcast")=3/9$  and  $𝑃("No"|"Overcast")=0/5$

In [ ]:
df.outlook

In [ ]:
y_vals = df[df.label.str.contains('Yes')].count()[0]
n_vals = df[df.label.str.contains('No')].count()[0]
f_vals = df.count()[0]
y_vals, n_vals, f_vals

In [ ]:
df.outlook.unique()

In [ ]:
sunny_y = df[df.outlook.str.contains('Sunny') & df.label.str.contains('Yes')].count()[0] 
sunny_n = df[df.outlook.str.contains('Sunny') & df.label.str.contains('No')].count()[0] 
overcast_y = df[df.outlook.str.contains('Overcast') & df.label.str.contains('Yes')].count()[0] 
overcast_n = df[df.outlook.str.contains('Overcast') & df.label.str.contains('No')].count()[0] 
rain_y = df[df.outlook.str.contains('Rain') & df.label.str.contains('Yes')].count()[0]
rain_n = df[df.outlook.str.contains('Rain') & df.label.str.contains('No')].count()[0]
print(f'sunny_y = {sunny_y}/{y_vals}, sunny_n = {sunny_n}/{n_vals}')
print(f'overcast_y = {overcast_y}/{y_vals}, overcast_n = {overcast_n}/{n_vals}')
print(f'rain_y = {rain_y}/{y_vals}, rain_y = {rain_y}/{n_vals}')

In [ ]:
def count_feat(col_name:str, col_val:str) -> int:
    return df[df[col_name].str.contains(col_val)].count()[0]

def count_cond(col_name:str, col_val:str, lab:str) -> int:
    return df[df[col_name].str.contains(col_val) & df.label.str.contains(lab)].count()[0]

def count_probs(col_name:str) -> int:
    col_vals = df[col_name].unique()
    for col_val in col_vals:
        val_y = count_cond(col_name, col_val, 'Yes')
        val_n = count_cond(col_name, col_val, 'No')
        val_f = count_feat(col_name, col_val)
        yield val_y, val_n, val_f, col_val
    

In [ ]:
col_vals = df.temp.unique()
temp_vals = [(ys, ns, fs, vls) for (ys, ns, fs, vls) in count_probs('temp')]
temp_vals

In [ ]:
col_vals = [(col_name, [(ys, ns, fs, vls) for (ys, ns, fs, vls) in count_probs(col_name)]) 
            for col_name in df.columns]
col_vals

In [ ]:
lns = ''
for col_val in col_vals:
    ln = f'{col_val[0]}: \n' + '\n'.join(f'P({nm}) = {f_v}, P({nm}|Yes) = {y_v}/{y_vals}, P({nm}|No) = {n_v}/{n_vals}'
                               for y_v, n_v, f_v, nm in col_val[1]) + '\n'
    lns += ln
    lns += '===============\n'
print(lns)

In [ ]:
model_vals = {col_name: {vls: (ys, ns, fs) for (ys, ns, fs, vls) in count_probs(col_name)} 
            for col_name in df.columns if col_name != 'label'}
model_vals

Outlook: "Sunny",
Temperature: "Cool",
Humidity: "High",
Wind: "Strong"


In [ ]:
out_v = model_vals['outlook']['Sunny']
tmp_v = model_vals['temp']['Cool']
hum_v = model_vals['humidity']['High']
wnd_v = model_vals['wind']['Strong']
yes_raw = (out_v[0] / y_vals) * (tmp_v[0] / y_vals) * (hum_v[0] / y_vals) * (wnd_v[0] / y_vals) * (y_vals / f_vals) 
no_raw = (out_v[1] / n_vals) * (tmp_v[1] / n_vals) * (hum_v[1] / n_vals) * (wnd_v[1] / n_vals) * (n_vals / f_vals)
yes_raw, no_raw

In [ ]:
p_x = (out_v[2] / f_vals) * (tmp_v[2] / f_vals) * (hum_v[2] / f_vals) * (wnd_v[2] / f_vals)
p_x

In [ ]:
yes_pred = yes_raw / p_x
no_pred = no_raw / p_x
print(f'yes_pred = {yes_pred}, no_pred = {no_pred}')

In [ ]:
def predict(x:tuple) -> tuple:
    out_v = model_vals['outlook'][x[0].capitalize()]
    tmp_v = model_vals['temp'][x[1].capitalize()]
    hum_v = model_vals['humidity'][x[2].capitalize()]
    wnd_v = model_vals['wind'][x[3].capitalize()]
    yes_raw = (out_v[0] / y_vals) * (tmp_v[0] / y_vals) * (hum_v[0] / y_vals) * (wnd_v[0] / y_vals) * (y_vals / f_vals) 
    no_raw = (out_v[1] / n_vals) * (tmp_v[1] / n_vals) * (hum_v[1] / n_vals) * (wnd_v[1] / n_vals) * (n_vals / f_vals)
    p_x = (out_v[2] / f_vals) * (tmp_v[2] / f_vals) * (hum_v[2] / f_vals) * (wnd_v[2] / f_vals)
    yes_pred = yes_raw / p_x
    no_pred = no_raw / p_x
    
    return yes_pred, no_pred

In [ ]:
x_vec = ('Sunny', 'Cool', 'High', 'Strong')
y_pr, n_pr = predict(x_vec)
print(f'yes_pred = {y_pr}, no_pred = {n_pr}')

## Analysis of algorithm

Learning a Naive Bayes classifier is just a matter of counting how many times each attribute co-occurs with each class


As you observed algorithm fits more for categorical variables rather than numerical. Also if sample distribution is representative.

#### Advantages
- It performs better on multi-class classification. 
- It performs better on smaller datasets. 
- If independence is hold, Naive Baye's performs better than Logistic regression on smaller datasets.
- It performs better for categorical variables, for numerical variables Gaussian distribution is preffered.

#### Disadvantages
- If categorical variable is unseen during the training then we get zero frequency problem and smoothing technique is used (Laplasian Smoothing)
- It is known as bad estimator, probabilities should not be tacken in consideration
- In real life features are not independent

Main question is how can we deal with zero frequencies. Laplassian smoothing is a little trick:
$$P(x_i|C_p) = \frac{count(x_i, C_p) + 1}{\sum_{j=1}^{n}(count(x_j, C_p) + 1)}$$
<br>
or
$$P(x_i|C_p) = \frac{count(x_i, C_p) + 1}{\sum_{j=1}^{n}count(x_j, C_p) + n}$$

## Theoretical Part

Recall conditional probability
$$p(C_k \mid x_1, \dots, x_n)$$
and Baye's theorem
$$p(C_k \mid \mathbf{x}) = \frac{p(C_k) \ p(\mathbf{x} \mid C_k)}{p(\mathbf{x})}$$

Recall that for given $C$ the probability $$P(A \mid C)$$ can be considered as usual probability mesure:
- $$P(\Omega \mid C) = 1$$
- $$P(A \mid C) \le 1$$
etc

The evens $A$ and $B$ are independent if $P(A\cap B) = P(A)P(B)$ or $P(A, B) = P(A)P(B)$ we can use this formula for conditional independence (implies from ebove property of conditional probability)
$$P(A \cap B | C) = P(A \mid C)P(B \mid C)$$

By the definition of conditional probability $P(A\cap B)= P(A \mid B)P(B)$
<br>
$$\begin{align}
P(A \mid B \cap C) = \frac{P(B \cap C \cap A)}{P(B \cap C)}= \frac{P(B \cap A \cap C)}{P(B \cap C)} = \\
\frac{P(A \cap B \mid C)P(C)}{P(B \cap C)} = \\
\frac{P(A \mid C)P(B \mid C)P(C)}{P(B \cap C)} = \\
\frac{P(A \mid C)P(B \mid C)P(C)}{P(B \mid C)P(C)} = P(A \mid C)
\end{align}$$
<br>
$$P(A| B \cap C) = P(A \mid B, C) = P(A\mid C)$$

$$\begin{align}
P(C_k, x_1, \dots, x_n) & = P(x_1, \dots, x_n, C_k) \\
                        & = P(x_1 \mid x_2, \dots, x_n, C_k) \ P(x_2, \dots, x_n, C_k) \\
                        & = P(x_1 \mid x_2, \dots, x_n, C_k) \ P(x_2 \mid x_3, \dots, x_n, C_k) \ P(x_3, \dots, x_n, C_k) \\
                        & = \dots \\
                        & = P(x_1 \mid x_2, \dots, x_n, C_k) \ P(x_2 \mid x_3, \dots, x_n, C_k) \dots   P(x_{n-1} \mid x_n, C_k) \ p(x_n \mid C_k) \ P(C_k) \\
\end{align}$$

In general
$$P(x_i \mid x_{i+1}, \dots ,x_{n}, C_k ) = P(x_i \mid C_k)$$

We have formula:
$$\begin{align}
P(C_k, x_1, \dots, x_n) & = \frac{P(x_1 \mid C_k)P(x_2 \mid C_k) \dots P(x_n \mid C_k)P(C_k)}{P(x)} \\
& = \frac{P(x_1 \mid C_k)P(x_2 \mid C_k) \dots P(x_n \mid C_k)P(C_k)}{P(x_1, x_2, \dots, x_n)} \\
&= \frac{P(x_1 \mid C_k)P(x_2 \mid C_k) \dots P(x_n \mid C_k)P(C_k)}{P(x_1)P(x_2) \dots P(x_n)} \\
\end{align}$$

Denominator is the same for all classes so classifier will decide by the formula:
$$\hat{y} = \underset{k \in \{1, \dots, K\}}{\operatorname{argmax}} \ P(C_k) \displaystyle\prod_{i=1}^n P(x_i \mid C_k)$$

## Different types of Naive Baye's

#### Multinomial naive Bayes

Let event occures with $(p_1, \dots, p_n)$ probabilities, then we have:
<br>
$$P(\mathbf{x} \mid C_k) = \frac{(\sum_i x_i)!}{\prod_i x_i !} \prod_i {p_{ki}}^{x_i}$$

Then our estimator with $\log$ becomes linear classifier:
$$
\begin{align}
\log p(C_k \mid \mathbf{x}) & \varpropto \log \left( p(C_k) \prod_{i=1}^n {p_{ki}}^{x_i} \right) \\
                       & = \log p(C_k) + \sum_{i=1}^n x_i \cdot \log p_{ki}                 \\
                       & = b + \mathbf{w}_k^\top \mathbf{x}
\end{align}
$$
<br>
where $b = \log p(C_k)$ and $w_{ki} = \log p_{ki}$

#### Bernoulli naive Bayes

When we have occurrence instead of frequency:
$$P(\mathbf{x} \mid C_k) = \prod_{i=1}^n p_{ki}^{x_i} (1 - p_{ki})^{(1-x_i)}$$

#### Gaussian naive Bayes

Let for some $x$ and class $C_k$ data is distrubuted with normal (Gaussian) distribution.
Compute mean and variance of $x$ for each class: $\mu_k$ and $\sigma_{k}^{2}$, then fop value $v$:
$$p(x=v \mid C_k)=\frac{1}{\sqrt{2\pi\sigma^2_k}}\,e^{ -\frac{(v-\mu_k)^2}{2\sigma^2_k} }$$

#### Excercise:
Try The approach for spam classifier

#### Excercise:
Classify <a href="https://archive.ics.uci.edu/ml/datasets/iris">Iris dataset</a> with Naive Baye's classifier

#### Excercise:
Classify <a href="https://archive.ics.uci.edu/ml/datasets/Wine">Wine dataset</a> with Naive Baye's classifier

#### Excercise:
Classify <a href="https://archive.ics.uci.edu/ml/datasets/Adult">Adult dataset</a> with Naive Baye's classifier